https://colab.research.google.com/github/bnsreenu/python_for_microscopists/blob/master/tips_tricks_38_Installing_conda_in_Google_Colab.ipynb#scrollTo=Hech3AARiLXp

In [ ]:
#Path and version numbers for python
!which python
!python --version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install quickumls
!pip install dill

In [ ]:
%cd '/content/drive/MyDrive/DL4HC Final Project/concept_annotation/'

/content/drive/.shortcut-targets-by-id/1Z6LHAVVC9fSl0B9pwGd6rlPzt1fN2UEW/DL4HC Final Project/concept_annotation


In [ ]:
from quickumls import QuickUMLS

In [ ]:
import pandas as pd 
    
# making dataframe 
data = pd.read_csv("/content/drive/MyDrive/DL4HC Final Project/concept_annotation/data/chunkssmall/1.csv") 
data.columns = [*data.columns[:-1], 'text'] # rename column and remove whitespace.
data['cui_set'] = None
data['term_set'] = None
# output the column names
data.columns.values

array(['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'cui_set', 'term_set'], dtype=object)

In [ ]:
for i in range(5):
    row = data.iloc[i]
    text_value = row['text']
    print(text_value)

 radiologic studies radiologic studies also included a chest ct which confirmed cavitary lesions in the left lung apex consistent with infectious processtuberculosis this also moderate sized left pleural effusionhead ct head ct showed no intracranial hemorrhage or mass effect but old infarction consistent with past medical historyabdominal ct abdominal ct showed lesions of t ten and sacrum most likely secondary to osteoporosis these can be followed by repeat imaging as an outpatient 
 history of present illness this is an eighty one year old female with a history of emphysema not on home o two who presents with three days of shortness of breath thought by her primary care doctor to be a copd flare two days prior to admission she was started on a prednisone taper and one day prior to admission she required oxygen at home in order to maintain oxygen saturation greater than ninety percent she has also been on levofloxacin and nebulizers and was not getting better and presented to the emer

In [ ]:
data_copy = data.iloc[:200].copy()

In [ ]:
import pandas as pd

TUIs = ["T020","T190","T049","T019","T047","T050","T033","T037","T048","T191","T046","T184","T038","T069","T068","T034","T070","T067","T043","T201","T045","T041","T044","T032","T040","T042","T039","T116","T195","T123","T122","T103","T120","T104","T200","T196","T126","T131","T125","T129","T130","T197","T114","T109","T121","T192","T127"]

# Load your data into a DataFrame

for idx, row in data.iterrows():
    line = row['text']  # Get the value in the "text" column for the current row

    cui_set = set()
    term_set = set()

    matches = matcher.match(line, best_match=True, ignore_syntax=False)

    for phrase_candidate in matches:
        # Find max
        max_similarity = 0
        for candidate in phrase_candidate:
            if candidate['similarity'] > max_similarity and set(candidate['semtypes']).intersection(TUIs):
                max_similarity = candidate['similarity']
        # Get preferred terms for that max
        if max_similarity >= 0:
            for candidate in phrase_candidate:
                if candidate['similarity'] == max_similarity:
                    if candidate['term'] not in term_set:
                        if candidate['cui'] not in cui_set:
                            cui_set.add(candidate['cui'])
                            term_set.add(candidate['term'])

    # Add cui_set and term_set to the DataFrame as new columns
    data.at[idx, 'cui_set'] = list(cui_set)
    data.at[idx, 'term_set'] = list(term_set)

    print(f"Processed row {idx + 1}")
    print(f"CUIs extracted: {data.at[idx, 'cui_set']}")

print(data)


In [ ]:
#  len(data. index)

4914

In [ ]:
data

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text,cui_set,term_set
0,174,22532,167853,2151-08-04,NaN,NaN,discharge summary,report,NaN,NaN,radiologic studies radiologic studies also in...,None,None
1,175,13702,107527,2118-06-14,NaN,NaN,discharge summary,report,NaN,NaN,history of present illness this is an eighty ...,None,None
2,176,13702,167118,2119-05-25,NaN,NaN,discharge summary,report,NaN,NaN,allergies amlodipineattending chief complaint...,None,None
3,177,13702,196489,2124-08-18,NaN,NaN,discharge summary,report,NaN,NaN,allergies amlodipineattending chief complaint...,None,None
4,178,26880,135453,2162-03-25,NaN,NaN,discharge summary,report,NaN,NaN,allergies patient recorded as having no known...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,5097,61791,176839,2187-05-07,NaN,NaN,discharge summary,report,NaN,NaN,allergies no known allergies adverse drug rea...,None,None
4910,5098,27841,149471,2120-01-27,NaN,NaN,discharge summary,report,NaN,NaN,allergies patient recorded as having no known...,None,None
4911,5099,6983,173511,2177-08-06,NaN,NaN,discharge summary,report,NaN,NaN,history of the present illness the patient is...,None,None
4912,5100,6983,151448,2177-08-21,NaN,NaN,discharge summary,report,NaN,NaN,history of present illness the patient is a s...,None,None
